# Jetpack Compose - Model-View-ViewModel (MVVM) - Podstawy

W tej aplikacji przyjrzymy się zastosowaniu architektury **MVVM**.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExdmllMXdiMmd0OW8wNGs3bXJucXJxYmNtZGpheGtueG5wbzZnZDduaCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/ff1yWpIVOu1FqfQA3Q/giphy.gif" width="200" />

**Model-View-ViewModel (MVVM)** to wzorzec architektoniczny stosowany w aplikacjach mobilnych (i nie tylko), który ma na celu oddzielenie logiki biznesowej od warstwy prezentacji.

W MVVM występują trzy główne komponenty:
- **Model** - Reprezentuje dane i logikę biznesową aplikacji. Model może zawierać struktury danych, metody dostępu do bazy danych, usługi sieciowe itp.
- **View** - Odpowiada za prezentację interfejsu użytkownika (UI). Może to być np. widok, przyciski, pola tekstowe, komponenty, ekrany itp. 
- **ViewModel** - Pośredniczy między Modelem a Widokiem. ViewModel zawiera dane, które mają być wyświetlane w widoku oraz metody, które obsługują interakcje użytkownika. ViewModel nie jest bezpośrednio zależny od Widoku - jest odizolowany od konkretnego interfejsu użytkownika.

Kilka korzyści wynikających z zastosowania wzorca MVVM:
- **Separacja logiki biznesowej od warstwy prezentacji** - Dzięki MVVM można utrzymać czytelność i organizację kodu, ponieważ logika biznesowa jest oddzielona od kodu odpowiedzialnego za wygląd i interakcję z użytkownikiem.
- **Testowalność** - Dzięki oddzieleniu ViewModelu od Widoku można łatwiej testować logikę biznesową, ponieważ ViewModel **nie jest bezpośrednio zależny** od interfejsu użytkownika. Można napisać testy jednostkowe dla ViewModelu, symulując różne scenariusze bez potrzeby uruchamiania całej aplikacji.
- **Ponowne wykorzystanie kodu** Dzięki MVVM można łatwiej ponownie wykorzystywać ViewModel w różnych miejscach aplikacji. Na przykład, ten sam ViewModel może być używany w różnych ekranach, które mają podobne wymagania, ale różnią się układem i wyglądem.
- **Łatwiejsze zarządzanie stanem** MVVM pomaga w zarządzaniu stanem aplikacji poprzez wprowadzenie jednokierunkowego przepływu danych. ViewModel dostarcza dane do Widoku, które je wyświetla, a wszelkie zmiany wprowadzane przez użytkownika są przekazywane z powrotem do ViewModelu.
- **Szybsze budowanie aplikacji** - Elementy MVVM są niezależne. Zmiany w pojedynczym elemencie wymagają preezebudowania tylko tego elementu, przykładowo, zmiana w klasie `ViewModel` będzie wymagała przekompilowania tylko tej klasy, a nie całego projektu.
- **Jednokierunkowy przepływ danych** - *Unidirectional data flow* (UDF) to wzorzec projektowy, w którym stan przepływa w dół, a zdarzenia przepływają w górę. Poprzez stosowanie jednokierunkowego przepływu danych, można odseparować komponenty odpowiedzialne za wyświetlanie stanu w interfejsie użytkownika od części aplikacji, które przechowują i zmieniają stan.

<img src="https://developer.android.com/static/images/jetpack/compose/state-unidirectional-flow.png" width="200" />

Wszystkie elementy architektury znamy z poprzednich przykładów, teraz napiszemy prostą aplikację wyświetlającą imiona i nazwiska użytkowników, ktorzy będą przechowywani w liście. Wykorzystamy również *dummy data* do zainicjowania listy.

### Model

Rozpocznijmy od modelu.

In [ ]:
data class User(val firstName: String, val lastName: String)

Mamy klasę `User` z dwoma polami reprezentującymi imię i nazwisko. Dodajmy `DataProvider` z danymi inicjującymi.

In [ ]:
object DataProvider {

    private val firstNames = listOf(
        "Adam", "Ewa", "Jan", "Anna", "Piotr", "Maria", "Tomasz", "Małgorzata", "Krzysztof", "Alicja",
        "Andrzej", "Joanna", "Michał", "Barbara", "Kamil", "Magdalena", "Robert", "Monika", "Mateusz", "Natalia"
    )

    private val lastNames = listOf(
        "Nowak", "Kowalski", "Wiśniewski", "Wójcik", "Kowalczyk", "Kamiński", "Lewandowski", "Zieliński", "Szymański",
        "Woźniak", "Dąbrowski", "Kozłowski", "Jankowski", "Mazur", "Kwiatkowski", "Krawczyk", "Piotrowski", "Grabowski",
        "Nowakowski", "Pawłowski"
    )

    val users = (0..40).map { User(firstNames.random(), lastNames.random())}
}

### ViewModel

W klasie `UserViewModel` przechowujemy listę wszystkich użytkowników jako `SnapshotStateList<User>`, który uzyskujemy wywołując metodę `mutableStateListOf`.

`SnapshotStateList` przechowuje aktualny stan listy i umożliwia dostęp do jej elementów oraz manipulację nimi. W przeciwieństwie do zwykłej Listy, umożliwia modyfikowanie zawartości listy.
- Możliwość odczytu - umożliwia odczytanie aktualnego stanu listy za pomocą właściwości `value`. Pozwala to na dostęp do elementów listy i ich wykorzystanie w interfejsie użytkownika.
- Możliwość zmiany - dostarcza metody, które umożliwiają dodawanie, usuwanie i modyfikowanie elementów listy. Gdy następują zmiany, `SnapshotStateList` automatycznie zapisuje nowy stan listy i informuje o tym zainteresowane komponenty (subskrybentów).
- Zachowanie niezmienności - zapewnia, że po dokonaniu modyfikacji na liście zostanie utworzony nowy obiekt listy, zamiast bezpośrednio modyfikować istniejący stan. Dzięki temu, współpracuje dobrze z reaktywnymi interfejsami użytkownika, gdzie zmiana stanu powoduje ponowne renderowanie komponentów.

In [ ]:
class UserViewModel : ViewModel() {
    private var _usersList = mutableStateListOf<User>()
    val usersList: List<User>
        get() = _usersList

    init {
        reinitialize()
    }

    fun addUser(user: User){
        _usersList.add(user)
        _usersList.sortBy { it.lastName }
    }

    fun reinitialize(){
        _usersList.clear()
        _usersList.addAll(DataProvider.users)
        _usersList.sortBy { it.lastName }
    }

    fun clear(){
        _usersList.clear()
    }
}

- `val usersList: List<User> get() = _usersList` - lista użytkowników, do której można uzyskać dostęp tylko do odczytu.
- `fun addUser(user: User)` - Metoda służy do dodawania nowego użytkownika do listy. 
- `fun reinitialize()` - Metoda służy do zresetowania zawartości listy użytkowników do wartości domyślnych.
- `fun clear()` - Metoda służy do wyczyszczenia całej listy użytkowników.

### View

Dodajmy `View` do aplikacji, czyli zdefiniujmy interfejs użytkownika. Funkcja `@Composable` `MainScreen` zawiera ui z dwoma polami edytowalnymi, listą użytkowników oraz trzema przyciskami odpowiedzialnymi za dodanie elementu, wyczyszczenie oraz reinicjalizację listy.

In [ ]:
@OptIn(ExperimentalMaterial3Api::class)
@Composable
fun MainScreen(){
    ...

    Column(modifier = Modifier.padding(2.dp)) {
        Row(
            modifier = Modifier.fillMaxWidth(),
            verticalAlignment = Alignment.CenterVertically
        ) {
            TextField(
                modifier = Modifier.padding(start = 4.dp).weight(1f),
                value = ...,
                onValueChange = { ... },
                label = { Text("First Name") }
            )

            TextField(
                modifier = Modifier.padding(start = 4.dp).weight(1f),
                value = ...,
                onValueChange = { ... },
                label = { Text("Last Name") }
            )

            Button(
                modifier = Modifier
                    .weight(.7f)
                    .padding(start = 4.dp, end = 4.dp),
                shape = RoundedCornerShape(8.dp),
                onClick = {
                    ...
                }
            ) {
                Text(text = "ADD")
            }
        }

        LazyColumn(modifier = Modifier.weight(1f)){
            items(viewModel.usersList.size){
                Text(
                    text = ...,
                    fontSize = 32.sp,
                    textAlign = TextAlign.Center,
                    modifier = Modifier
                        .fillMaxWidth()
                        .padding(2.dp)
                )
            }
        }

        Button(
            modifier = Modifier
                .fillMaxWidth()
                .padding(start = 2.dp, end = 4.dp),
            shape = RoundedCornerShape(8.dp),
            onClick = { ... }
        ) {
            Text(text = "CLEAR")
        }

        Button(
            modifier = Modifier
                .fillMaxWidth()
                .padding(start = 2.dp, end = 4.dp),
            shape = RoundedCornerShape(8.dp),
            onClick = { ... }
        ) {
            Text(text = "RESET")
        }

    }
}

Potrzebujemy dwie zmienne przechowujące stan pól edytowalnych.

In [ ]:
var firstName by remember { mutableStateOf("") }
var lastName by remember { mutableStateOf("") }

Oraz wartość przechowującą instancję `UserViewModel`

In [ ]:
val viewModel: UserViewModel = viewModel()

Zmodyfikujmy edytowalne pola `TextField`

In [ ]:
TextField(
    modifier = Modifier.padding(start = 4.dp).weight(1f),
    value = firstName,
    onValueChange = { firstName = it },
    label = { Text("First Name") }
)

TextField(
    modifier = Modifier.padding(start = 4.dp).weight(1f),
    value = lastName,
    onValueChange = { lastName = it },
    label = { Text("Last Name") }
)

Przy zmianie wartości pola, jego stan jest zapisywany w zmiennych `firstName` oraz `lastName`

Dodajmy `onClick` przycisku odpowiedzialnego za dodanie nowego użytkownika.

In [ ]:
onClick = {
    if ("$firstName$lastName".isNotBlank()) {
        viewModel.addUser(User(firstName, lastName))
        firstName = ""
        lastName = ""
    }
}

Sprawdzam czy pola nie są puste, następnie wywołuję metodę `add` klasy `UserVViewModel`, tworzę nowego użytkownika i dodaję go do listy. Na koniec resetuję wartości `firstName` i `lastName`.

Wyświetlmy listę wszystkich użytkowników.

In [ ]:
LazyColumn(modifier = Modifier.weight(1f)){
    items(viewModel.usersList.size){
        Text(
            text = "${viewModel.usersList[it].firstName} ${viewModel.usersList[it].lastName}", // jedno pole na którym wyświetlam imię i nazwisko
            fontSize = 32.sp,
            textAlign = TextAlign.Center,
            modifier = Modifier
                .fillMaxWidth()
                .padding(2.dp)
        )
    }
}

Na koniec dodajmy obsługę `onClick` przycisków odpowiedzialnych za czyszczenie oraz reinicjalizację danych.

In [ ]:
Button(
    modifier = Modifier
        .fillMaxWidth()
        .padding(start = 2.dp, end = 4.dp),
    shape = RoundedCornerShape(8.dp),
    onClick = { viewModel.clear() } // czyszczenie
) {
    Text(text = "CLEAR")
}

Button(
    modifier = Modifier
        .fillMaxWidth()
        .padding(start = 2.dp, end = 4.dp),
    shape = RoundedCornerShape(8.dp),
    onClick = { viewModel.reinitialize() } // reinicjalizacja
) {
    Text(text = "RESET")
}

Możemy przetestować aplikację.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExdmllMXdiMmd0OW8wNGs3bXJucXJxYmNtZGpheGtueG5wbzZnZDduaCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/ff1yWpIVOu1FqfQA3Q/giphy.gif" width="200" />